In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-12"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9108,2024-06-12,Nova Zelândia Nbl,04:30,Otago Nuggets,Taranaki Airs,5.35,1.13,185.5,1.86,1.84,0.0,0.00,0.00,YXGX2Lia,0.186916,0.884956,0.537634,0.543478,0.071872,323.768,281.792961,0.870355,1.2,1.643168,1.369306,209.52,3.544,3.005700,0.848109,-4.0,111.438,6.415736,0.057572,2.4,1.341641,0.559017,121.55,1.120,0.176068,0.157204,88.0,108,85,1.94,1.43,222.738,148.720,0.920985,0.007644,NaN,2.15,0.430,10.116279,0.000000,0.0,0.000000,-0.93,-0.186,-0.698925,0.000000,0.0,0.000000
9109,2024-06-12,Itália Série A2,16:00,Trieste,Cantu,1.67,2.12,152.5,1.87,1.79,-3.5,2.01,2.19,f7kVkO4L,0.598802,0.471698,0.534759,0.558659,0.070501,166.498,49.995436,0.300276,2.4,1.341641,0.559017,104.39,2.056,0.570596,0.277527,48.0,141.086,45.233841,0.320612,1.2,1.643168,1.369306,198.32,1.930,0.499149,0.258627,-29.0,73,74,1.43,2.68,114.982,144.328,0.167915,0.030912,0.060609,-0.91,-0.182,-3.681319,0.751011,0.6,-0.151011,1.19,0.238,4.705882,0.607864,0.5,-0.107864
9110,2024-06-12,Lituânia Lkl,12:50,Rytas,Zalgiris Kaunas,2.25,1.61,170.5,1.87,1.87,0.0,0.00,0.00,Y3FwXeJG,0.444444,0.621118,0.534759,0.534759,0.065562,181.322,86.330970,0.476120,1.8,1.643168,0.912871,298.89,2.032,1.028747,0.506273,3.0,114.404,18.762097,0.163999,2.4,1.341641,0.559017,114.66,1.252,0.180194,0.143925,31.0,81,91,3.69,1.26,143.590,123.844,0.234481,0.000000,NaN,1.13,0.226,5.530973,0.748108,0.8,0.051892,-2.47,-0.494,-1.234818,0.831367,0.7,-0.131367
9111,2024-06-12,Taiwan P. League+,08:00,Taoyuan Pauian Pilots,New Taipei Kings,1.66,2.13,180.5,1.80,1.88,0.0,0.00,0.00,OUo2atYi,0.602410,0.469484,0.555556,0.531915,0.071893,136.208,12.015543,0.088215,1.8,1.643168,0.912871,140.14,1.382,0.120291,0.087041,15.0,182.332,61.201463,0.335659,1.2,1.643168,1.369306,139.08,2.006,0.702410,0.350155,30.0,98,76,1.43,1.83,124.144,193.534,0.175377,0.030744,NaN,0.22,0.044,15.000000,0.693249,0.9,0.206751,-1.65,-0.330,-3.424242,0.546198,0.5,-0.046198
9112,2024-06-12,Grécia Liga De Basquete,15:15,Olympiakos,Panathinaikos,1.62,2.31,154.5,1.88,1.90,0.0,0.00,0.00,tGaVRnVq,0.617284,0.432900,0.531915,0.526316,0.050184,137.852,57.579791,0.417693,2.4,1.341641,0.559017,234.08,1.630,0.835883,0.512812,55.0,125.148,44.757828,0.357639,2.4,1.341641,0.559017,114.54,1.438,0.527229,0.366640,37.0,76,83,3.08,1.38,104.110,120.732,0.248297,0.007483,NaN,0.92,0.184,3.369565,0.924486,1.0,0.075514,-0.26,-0.052,-25.192308,0.855315,0.9,0.044685
9113,2024-06-12,Colômbia Lbp,21:30,Caribbean Storm Islands,Motilones del Norte,1.42,2.72,154.5,1.85,1.85,0.0,0.00,0.00,8tWVl1hR,0.704225,0.367647,0.540541,0.540541,0.071872,188.828,141.683611,0.750332,1.8,1.643168,0.912871,127.92,2.578,2.066028,0.801407,13.0,123.714,29.929343,0.241924,3.0,0.000000,0.000000,175.77,1.404,0.439352,0.312929,40.0,78,81,1.64,2.17,122.428,147.960,0.444077,0.000000,NaN,0.52,0.104,4.038462,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9114,2024-06-12,México Cibacopa,23:00,Angeles CDMX,Astros,2.45,1.51,194.5,1.81,1.89,0.0,0.00,0.00,GhdEEHhc,0.408163,0.662252,0.552486,0.529101,0.070415,201.820,57.982444,0.287298,1.8,1.643168,0.912871,285.19,2.134,0.896370,0.420042,-1.0,133.524,14.381854,0.107710,2.4,1.341641,0.559017,136.08,1.258,0.089275,0.070966,75.0,79,108,3.61,1.26,144.506,146.532,0.335697,0.030578,NaN,0.50,0.100,14.500000,0.690213,0.5,-0.190213,0.60,0.120,4.25

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9109,16:00,Itália Série A2,Trieste,Cantu,1.67,Back Home
